In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
from streaming.encounter_job import EncounterJob

In [4]:
job = EncounterJob()

In [ ]:
job.run()

Encounter Streaming Job Started at = 2020-03-17 09:51:39.835012
---Micro-Batch--- 
Starting calculations for flat_enc_obs_orders Tue Mar 17 12:52:05 2020
CDC: Patient IDs  [16, 48, 64, 80, 96, 112, 144, 160, 224, 240, 256, 288, 304, 320, 336, 352, 400, 432, 448, 464, 496, 192, 32, 272, 384, 416, 368, 208, 17, 49, 113, 129, 209, 257, 273, 305, 337, 385, 401, 433, 449, 465, 193, 289, 321, 417, 81, 161, 177, 353, 33, 97, 145, 225, 241, 497, 369, 481, 18, 34, 50, 66, 114, 130, 162, 194, 226, 242, 290, 306, 322, 370, 418, 450, 482, 498, 82, 338, 434, 210, 274, 354, 466, 146, 19, 83, 147, 163, 179, 195, 211, 259, 275, 291, 323, 339, 355, 371, 403, 435, 467, 483, 99, 499, 227, 307, 51, 67, 35, 387, 419, 451, 243, 115, 36, 116, 148, 164, 180, 212, 228, 244, 260, 276, 308, 340, 372, 388, 404, 420, 436, 468, 20, 132, 52, 484, 84, 100, 324, 356, 452, 21, 69, 101, 133, 165, 229, 245, 261, 277, 293, 373, 405, 437, 453, 85, 309, 325, 357, 389, 421, 485, 53, 181, 197, 469, 213, 117, 501, 37, 38, 54, 

+------------+----------+-----------+--------+-------------------+--------------+-----+------+----------+--------------------+-------------+----------+--------------------+------+
|encounter_id|patient_id|location_id|visit_id| encounter_datetime|encounter_type| dead|gender|death_date|        patient_uuid|visit_type_id| birthdate|                 obs|orders|
+------------+----------+-----------+--------+-------------------+--------------+-----+------+----------+--------------------+-------------+----------+--------------------+------+
|         496|        45|          1|     212|2017-01-02 10:11:27|             5|false|     F|      null|b93fe5b8-9d85-454...|            1|1953-11-21|[{"obs_id":4120,"...|  null|
|         833|        74|          7|     353|2015-04-18 09:26:12|             2|false|     F|      null|c977b8e6-7953-412...|            1|1990-12-08|[{"obs_id":6858,"...|  null|
|        1342|       117|          1|     568|2016-03-07 10:31:26|             5|false|     F|      

Check the status: http://localhost:4040/streaming/